In [21]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import random
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, LSTM, RepeatVector, TimeDistributed

In [22]:
# Parámetros de ajuste
data = 'train-mod.csv'
kval = 7
itertot = 40
sigma = 1.2
itergd = 300

In [23]:
def transforminput(param, center):
	newinput = np.zeros((len(param), len(center))).astype('float32')
	for i in range(len(param)):
		for j in range(len(center)):
			newinput[i,j] = np.exp(-(np.sum((param[i] - center[j])**2.0)**0.5) / sigma**2.0)
	return newinput

In [24]:
## Observad como en este ejemplo, es distinta la implementacion 
## al ejemplo del ajuste a la función del guión directamente se utiliza 
## una capa Dense y se entrena utilizando lo que proporciona keras 
## tomando como entradas las salidas del kmedias


def generatemodel(numparam):
	model = Sequential()
	model.add(Dense(1, input_dim=numparam, activation='sigmoid'))
#	model.add(Dense(10, activation='sigmoid'))
#	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model



In [25]:
dataread = np.genfromtxt(data, delimiter=',')[1:,1:]

alldata = []
for i in range(len(dataread)):
	if np.isnan(dataread[i,-2]):
		continue
	alldata.append(dataread[i])

alldata = np.asarray(alldata)



In [26]:
# Partición de Datos en Entrenamiento y Test
trainparam = alldata[:600,1:]
trainlabel = alldata[:600,0]

testparam = alldata[600:,1:]
testlabel = alldata[600:,0]

In [27]:
###############
#Normalizacion de los Datos#
###############

std = np.zeros((len(trainparam[0]))).astype('float32')
rata = np.zeros((len(trainparam[0]))).astype('float32')
trainparamnorm = np.zeros(np.shape(trainparam))
testparamnorm = np.zeros(np.shape(testparam))
for i in range(len(trainparam[0])):
	std[i] = np.std(trainparam[:,i])
	rata[i] = np.mean(trainparam[:,i])
	trainparamnorm[:,i] = (trainparam[:,i] - rata[i]) / std[i]
	testparamnorm[:,i] = (testparam[:,i] - rata[i]) / std[i]

In [16]:
###############
#K-mean#
###############

#Inicializar kmean
kmean = np.zeros((kval, len(trainparamnorm[0])))

for i in range(kval):
	for j in range(len(kmean[0])):
		kmean[i,j] = random.uniform(min(trainparamnorm[:,j]),max(trainparamnorm[:,j]))

#Bucles del algoritmo
distmin = np.zeros((len(trainparamnorm)))
for i in range(itertot):
	print ('Iteracion: ', i)
	for j in range(len(distmin)):
		#determine euclid distance
		distall = np.sum((trainparamnorm[j] - kmean)**2.0, axis=1)**0.5
		distmin[j] = np.argmin(distall)

	#Buscar nuevo k-medias
	for j in range(kval):
		clust = []
		for k in range(len(distmin)):
			if distmin[k] == j:
				clust.append(trainparamnorm[k])
		if len(clust) > 0:
			kmean[j] = np.mean(np.asarray(clust), axis=0)

#Transformar nuestra entrada
newinput = transforminput(trainparamnorm, kmean)

print (trainlabel)

Iteracion:  0
Iteracion:  1
Iteracion:  2
Iteracion:  3
Iteracion:  4
Iteracion:  5
Iteracion:  6
Iteracion:  7
Iteracion:  8
Iteracion:  9
Iteracion:  10
Iteracion:  11
Iteracion:  12
Iteracion:  13
Iteracion:  14
Iteracion:  15
Iteracion:  16
Iteracion:  17
Iteracion:  18
Iteracion:  19
Iteracion:  20
Iteracion:  21
Iteracion:  22
Iteracion:  23
Iteracion:  24
Iteracion:  25
Iteracion:  26
Iteracion:  27
Iteracion:  28
Iteracion:  29
Iteracion:  30
Iteracion:  31
Iteracion:  32
Iteracion:  33
Iteracion:  34
Iteracion:  35
Iteracion:  36
Iteracion:  37
Iteracion:  38
Iteracion:  39
[0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0.
 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0.
 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0.

In [17]:
kval


7

In [18]:
##########################
#Gradiente Descendiente#
##########################

mod = generatemodel(kval)
mod.fit(newinput, trainlabel, batch_size=20, epochs=itergd, verbose=1, shuffle=True)		



2022-03-09 19:14:07.437583: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-09 19:14:07.439480: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Epoch 1/300
30/30 [==============================] - 0s 2ms/step - loss: 0.6494 - accuracy: 0.5850
Epoch 2/300
30/30 [==============================] - 0s 2ms/step - loss: 0.6475 - accuracy: 0.5867
Epoch 3/300
30/30 [==============================] - 0s 2ms/step - loss: 0.6458 - accuracy: 0.5900
Epoch 4/300
30/30 [==============================] - 0s 1ms/step - loss: 0.6437 - accuracy: 0.5867
Epoch 5/300
30/30 [==============================] - 0s 1ms/step - loss: 0.6419 - accuracy: 0.5867
Epoch 6/300
30/30 [==============================] - 0s 1ms/step - loss: 0.6402 - accuracy: 0.5867
Epoch 7/300
30/30 [==============================] - 0s 1ms/step - loss: 0.6384 - accuracy: 0.5850
Epoch 8/300
30/30 [==============================] - 0s 1ms/step - loss: 0.6367 - accuracy: 0.5867
Epoch 9/300
30/30 [==============================] - 0s 1ms/step - loss: 0.6350 - accuracy: 0.5883
Epoch 10/300
30/30 [==============================] - 0s 1ms/step - loss: 0.6333 - accuracy: 0.5850
Epoch 11/

30/30 [==============================] - 0s 1ms/step - loss: 0.5167 - accuracy: 0.7783
Epoch 163/300
30/30 [==============================] - 0s 1ms/step - loss: 0.5164 - accuracy: 0.7783
Epoch 164/300
30/30 [==============================] - 0s 1ms/step - loss: 0.5161 - accuracy: 0.7783
Epoch 165/300
30/30 [==============================] - 0s 979us/step - loss: 0.5158 - accuracy: 0.7783
Epoch 166/300
30/30 [==============================] - 0s 977us/step - loss: 0.5156 - accuracy: 0.7783
Epoch 167/300
30/30 [==============================] - 0s 924us/step - loss: 0.5153 - accuracy: 0.7783
Epoch 168/300
30/30 [==============================] - 0s 907us/step - loss: 0.5151 - accuracy: 0.7783
Epoch 169/300
30/30 [==============================] - 0s 857us/step - loss: 0.5148 - accuracy: 0.7783
Epoch 170/300
30/30 [==============================] - 0s 942us/step - loss: 0.5145 - accuracy: 0.7783
Epoch 171/300
30/30 [==============================] - 0s 949us/step - loss: 0.5143 - accurac

30/30 [==============================] - 0s 902us/step - loss: 0.5015 - accuracy: 0.7733
Epoch 243/300
30/30 [==============================] - 0s 1ms/step - loss: 0.5014 - accuracy: 0.7733
Epoch 244/300
30/30 [==============================] - 0s 884us/step - loss: 0.5012 - accuracy: 0.7733
Epoch 245/300
30/30 [==============================] - 0s 867us/step - loss: 0.5011 - accuracy: 0.7733
Epoch 246/300
30/30 [==============================] - 0s 832us/step - loss: 0.5010 - accuracy: 0.7717
Epoch 247/300
30/30 [==============================] - 0s 856us/step - loss: 0.5009 - accuracy: 0.7717
Epoch 248/300
30/30 [==============================] - 0s 831us/step - loss: 0.5009 - accuracy: 0.7717
Epoch 249/300
30/30 [==============================] - 0s 776us/step - loss: 0.5006 - accuracy: 0.7733
Epoch 250/300
30/30 [==============================] - 0s 970us/step - loss: 0.5005 - accuracy: 0.7733
Epoch 251/300
30/30 [==============================] - 0s 971us/step - loss: 0.5004 - acc

In [19]:
##################
#Etapa de Predicción#
##################

#Transformar los datos de Test
newinputtest = transforminput(testparamnorm, kmean)

lifeprob = mod.predict(newinputtest)

#######################
#Determinar el Rendimiento#
#######################

#Determinar precisión Binner
binpred = np.zeros((len(lifeprob)))
for i in range(len(lifeprob)):
	if lifeprob[i] > 0.5:
		binpred[i] = 1.

score = 0
for i in range(len(testlabel)):
	if binpred[i] == testlabel[i]:
		score += 1
accbin = float(score) / float(len(testlabel))

#determinar la puntuacion brier
brierscore = 0
for i in range(len(testlabel)):
	brierscore += (testlabel[i] - lifeprob[i])**2.0
brierscore = brierscore / float(len(testlabel))

for i in range(len(testlabel)):
	print (lifeprob[i], testlabel[i])
print (accbin, brierscore[0])

[0.09550494] 0.0
[0.62897617] 1.0
[0.42929125] 1.0
[0.0891808] 0.0
[0.25458258] 0.0
[0.1327532] 0.0
[0.8926927] 1.0
[0.19731227] 0.0
[0.12372565] 1.0
[0.9421971] 1.0
[0.17479336] 0.0
[0.7018032] 1.0
[0.62383753] 0.0
[0.11519521] 0.0
[0.08829582] 0.0
[0.2530388] 0.0
[0.48902896] 0.0
[0.51250875] 1.0
[0.14766517] 0.0
[0.61099803] 1.0
[0.7953234] 1.0
[0.63591063] 1.0
[0.7861452] 1.0
[0.41917518] 0.0
[0.08686182] 0.0
[0.0866417] 0.0
[0.6558992] 1.0
[0.30229276] 0.0
[0.38105184] 1.0
[0.43478888] 0.0
[0.2873422] 0.0
[0.08597028] 0.0
[0.20633313] 0.0
[0.5951963] 1.0
[0.6212901] 1.0
[0.10075018] 0.0
[0.6444087] 0.0
[0.19985989] 0.0
[0.69729733] 1.0
[0.6346585] 1.0
[0.38313752] 1.0
[0.08679041] 1.0
[0.10755739] 0.0
[0.34975937] 0.0
[0.6562885] 0.0
[0.20633313] 0.0
[0.7869762] 1.0
[0.08473486] 0.0
[0.18401176] 0.0
[0.20074096] 0.0
[0.62567776] 0.0
[0.1036641] 0.0
[0.66292423] 0.0
[0.2264517] 0.0
[0.21463385] 0.0
[0.27305615] 0.0
[0.7248249] 1.0
[0.08518025] 1.0
[0.35422772] 0.0
[0.6543464] 1.0
[